### 이상치 데이터 처리
- 피쳐/특성 데이터 값의 범위를 벗어나는 데이터를 의미
- 종류 : 잘못 입력된 경우, 혹은 자연으로 생성된 데이터
    - 잘못 입력된 데이터 : 입력 시의 오타 ===> 제거 또는 대체
    - 자연 생성 데이터 : 연봉, 키, 몸무게, 나이 ===> 분리해서 전용 모델/ 식별용 피쳐
- 탐지
    - 시각화 확인
    - 데이터의 분포를 확인 : Z-score /IQR(Turkey Fences)
- 처리

### (1) 모듈 로딩 및 데이터 준비 <hr>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
# 임의의 데이터
dataDF = pd.DataFrame({"num":[1,2,3,1,2,3,1,2,3,1,15]})
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   num     11 non-null     int64
dtypes: int64(1)
memory usage: 216.0 bytes


### (2) 데이터 전처리 <hr>

#### (2-1) 데이터 확인

In [ ]:
# 시각화로 데이터 분포 확인
